# Arkitect Cookbook: 从零开始构建您的第一个 AI Agent

欢迎来到 Arkitect 框架！这个 Cookbook 将引导您从基础概念开始，逐步构建一个功能完整的 AI Agent。

## 学习目标

通过这个 Cookbook，您将学会：

1. **理解 Context**：掌握 Arkitect 的核心概念 - 对话状态管理器
2. **构建 Agent**：学会如何将 Context 配置为特定任务的智能代理
3. **集成工具**：为您的 Agent 添加自定义功能
4. **精确控制**：通过参数精确调整 Agent 的行为模式
5. **调试观察**：使用 Hooks 深入了解 Agent 的内部工作机制

让我们开始这个令人兴奋的旅程！

---


## Arkitect Cookbook 目录

1. 第 0 章：环境准备与设置
    - 安装依赖
    - 配置 API Key
    - 导入必要的模块
    - 框架核心概念预览
2. 第 1 章：核心概念 - Context（对话管理器）
    - 创建您的第一个 Context
    - Context 的"记忆"能力
    - ArkMessage：消息的数据结构
    - State：对话状态的管理者
    - System Prompt 的威力
    - 第 1 章总结
3. 第 2 章：构建您的第一个 Agent
    - 工具系统的核心概念
    - 定义工具函数
    - 将函数转换为 ChatCompletionTool
    - Agent 的创建与工具集成
    - 工具调用与流式响应
    - ToolChunk：流式响应中的工具调用
    - ContextInterruption：异常处理机制
    - 第 2 章总结
4. 第 3 章：精确控制 Agent 行为 - ArkChatParameters
    - 参数控制的核心价值
    - ArkChatParameters：参数控制的核心
    - 参数实验与优化
    - 综合参数控制实验
    - 特殊参数演示：thinking
    - ArkChatRequest：请求的完整生命周期
    - ArkChatResponse：响应和使用统计
    - 第 3 章总结
5. 第 4 章：高级调试 - 使用 Hooks 观察 Agent
    - Hooks 的核心价值
    - 四种主要 Hook 类型
    - 创建和应用自定义 Hooks
    - 性能监控与异常处理 Hook
    - Hook 在自动化测试中的应用
    - Hook 最佳实践
    - 第 4 章总结
6. Q&A：常见问题和解决方案
7. 结语：您的 Arkitect 之旅

---

## 第 0 章：环境准备与设置

在开始构建您的第一个 Agent 之前，让我们确保您的环境已经正确配置。

### 步骤 1：安装依赖

首先，让我们安装 Arkitect 框架及其必要的依赖项：


In [ ]:
# 安装 Arkitect 框架
# 如果您还没有安装，请取消注释下面的行
# !pip install arkitect

import os
import sys

# 设置项目路径
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

# 验证安装
try:
    import arkitect

    print("Arkitect 框架已成功安装！")
    print(
        f"版本：{
            arkitect.__version__ if hasattr(arkitect, '__version__') else '开发版本'
        }"
    )
except ImportError:
    print("请先安装 Arkitect 框架：pip install arkitect")
    raise

Arkitect 框架已成功安装！
版本：


### 步骤 2：配置 API Key

为了使用 Arkitect 框架，您需要配置您的 API Key。这里我们提供一个安全的方式来设置您的 API Key：


In [ ]:
import getpass

# 检查是否已经设置了 API Key
if "ARK_API_KEY" in os.environ and os.environ["ARK_API_KEY"] != "YOUR_API_KEY_HERE":
    print("✅ API Key 已配置")
    print(f"🔑 当前 API Key 前缀：{os.environ.get('ARK_API_KEY', 'Not Set')[:10]}...")
else:
    print("🔐 请输入您的 API Key：")
    api_key = getpass.getpass("API Key: ")

    # 基本验证 - 确保不是默认占位符
    if api_key.strip() == "" or api_key == "YOUR_API_KEY_HERE":
        print("❌ 请输入有效的 API Key")
        raise ValueError("无效的 API Key")

    # 设置环境变量
    os.environ["ARK_API_KEY"] = api_key
    print("✅ API Key 已成功设置！")
    print(f"🔑 当前 API Key 前缀：{os.environ.get('ARK_API_KEY', 'Not Set')[:10]}...")

### 步骤 3：导入必要的模块

让我们导入本 Cookbook 中将要使用的核心模块：


In [5]:
import logging
from typing import Any, Optional

# 导入 Arkitect 核心组件
from arkitect.core.component.context.context import Context

# 导入 Hooks 相关类（用于高级调试）
from arkitect.core.component.context.hooks import (
    PostLLMCallHook,
    PostToolCallHook,
    PreLLMCallHook,
    PreToolCallHook,
)
from arkitect.core.component.context.model import State

# 导入用于自定义 Agent 行为的参数类
from arkitect.types.llm.model import (
    ArkChatParameters,
    ArkMessage,
    ChatCompletionTool,
)

# 静默HTTP相关的日志输出，避免干扰教程体验
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("urllib3.connectionpool").setLevel(logging.WARNING)
logging.getLogger("volcenginesdkarkruntime").setLevel(logging.WARNING)

print("✅ 所有必要的模块已成功导入！")
print("🚀 现在您可以开始构建您的第一个 Agent 了！")

✅ 所有必要的模块已成功导入！
🚀 现在您可以开始构建您的第一个 Agent 了！


### 步骤 4：框架核心概念预览

在深入学习之前，让我们先了解一下 Arkitect 框架的核心概念：

#### 🧠 **Context（对话管理器）**
- 框架的核心组件，管理对话状态和历史
- 自动处理多轮对话，无需手动管理消息历史
- 作为应用程序与大语言模型之间的桥梁

#### 📝 **ArkMessage（消息对象）**
- 表示对话中的单个消息
- 包含角色（user/assistant/system/tool）和内容
- 是 Context 内部存储和处理消息的基本单位

#### 🛠️ **ChatCompletionTool（工具系统）**
- 将 Python 函数转换为 LLM 可理解的工具描述
- 支持自动参数解析和函数调用
- 让 Agent 能够执行实际的操作

#### ⚙️ **ArkChatParameters（参数控制）**
- 精确控制 LLM 的行为（温度、最大token数等）
- 支持思考模式、停止词等高级功能
- 让您能够针对不同任务优化 Agent 性能

#### 🔍 **Hooks（调试观察）**
- 提供 Agent 执行过程的深度观察能力
- 支持性能监控、错误处理和自定义逻辑
- 是调试和优化 Agent 的重要工具

让我们通过一个简单的例子来预览这些概念：


In [ ]:
async def preview_core_concepts():
    """快速预览 Arkitect 框架的核心概念"""

    print("🎯 Arkitect 框架核心概念预览")
    print("=" * 50)

    # 1. 展示 ArkMessage 类
    print("\n📝 ArkMessage 类示例：")
    sample_message = ArkMessage(role="user", content="Hello, Arkitect!")
    print(f"   类型：{type(sample_message).__name__}")
    print(f"   角色：{sample_message.role}")
    print(f"   内容：{sample_message.content}")

    # 2. 展示 ChatCompletionTool 类
    print("\n🛠️ ChatCompletionTool 类示例：")

    def sample_tool(message: str) -> str:
        """一个示例工具函数"""
        return f"处理消息：{message}"

    tool = ChatCompletionTool.from_function(sample_tool)
    print(f"   工具名称：{tool.function.name}")
    print(f"   工具描述：{tool.function.description}")
    print(f"   参数结构：{list(tool.function.parameters.get('properties', {}).keys())}")

    # 3. 展示 ArkChatParameters 类
    print("\n⚙️ ArkChatParameters 类示例：")
    params = ArkChatParameters(temperature=0.7, max_tokens=100)
    print(f"   温度设置：{params.temperature}")
    print(f"   最大token：{params.max_tokens}")
    print(f"   参数类型：{type(params).__name__}")

    # 4. 展示 Context 类（不初始化，只展示结构）
    print("\n🧠 Context 类概览：")
    print("   - 管理对话状态和历史")
    print("   - 支持工具集成")
    print("   - 提供参数控制")
    print("   - 支持 Hook 系统")

    print("\n✨ 接下来我们将深入学习每个概念的详细用法！")
    print("=" * 50)


# 运行预览
await preview_core_concepts()

🎯 Arkitect 框架核心概念预览

📝 ArkMessage 类示例：
   类型：ArkMessage
   角色：user
   内容：Hello, Arkitect!

🛠️ ChatCompletionTool 类示例：
   工具名称：sample_tool
   工具描述：一个示例工具函数
   参数结构：['message']

⚙️ ArkChatParameters 类示例：
   温度设置：0.7
   最大token：100
   参数类型：ArkChatParameters

🧠 Context 类概览：
   - 管理对话状态和历史
   - 支持工具集成
   - 提供参数控制
   - 支持 Hook 系统

✨ 接下来我们将深入学习每个概念的详细用法！


---

## 第 1 章：核心概念 - Context（对话管理器）

现在让我们深入了解 Arkitect 的核心概念：**Context**。

### 什么是 Context？

Context 是 Arkitect 框架的核心组件，它充当一个智能的"对话管理器"。想象一下，Context 就像一个有记忆的助手，它能够：

- 🧠 **记住整个对话历史**：自动跟踪用户和 AI 之间的所有交互
- 🔄 **管理多轮对话**：无需手动管理对话状态，Context 会自动处理
- 🤖 **连接 AI 模型**：作为您的应用程序与大语言模型之间的桥梁
- 📊 **维护状态信息**：通过内部的 State 对象管理所有会话数据

让我们通过一个简单的例子来看看 Context 是如何工作的：


### 示例 1：创建您的第一个 Context

让我们创建一个基本的 Context 实例并进行第一次对话：


In [7]:
async def create_first_context():
    """演示基本的 Context 使用方式"""

    # 步骤 1：创建一个 Context 实例
    # Context 需要指定要使用的模型
    MODEL = "doubao-seed-1-6-flash-250615"  # 您可以根据需要更换模型

    print("🤖 创建第一个 Context...")
    context = Context(model=MODEL)

    # 步骤 2：初始化 Context
    # 这会建立与模型的连接
    await context.init()
    print(f"✅ Context 已初始化，使用模型：{MODEL}")

    # 步骤 3：发送第一条消息
    print("\n🗣️ 开始第一轮对话...")

    first_message = "我是一个会飞的西兰花，和我打个招呼吧？"

    # 使用 Context 发送消息并获取响应
    completion = await context.completions.create(
        [{"role": "user", "content": first_message}],
        stream=True,  # 启用流式响应，可以实时看到输出
    )

    print(f"👤 用户: {first_message}")
    print("🤖 AI: ", end="")

    # 处理流式响应
    async for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")

    print("\n" + "=" * 50)

    return context  # 返回 context 以便后续使用


# 运行第一次对话
context = await create_first_context()

🤖 创建第一个 Context...
2025-08-05 19:36:47 [debug    ] singleton class initialized    name=ClientPool
✅ Context 已初始化，使用模型：doubao-seed-1-6-flash-250615

🗣️ 开始第一轮对话...
👤 用户: 我是一个会飞的西兰花，和我打个招呼吧？
🤖 AI: 哇哦，会飞的西兰花你好呀！感觉你好特别呀，快给我讲讲你飞起来的奇妙体验呗~


### 示例 2：Context 的"记忆"能力

现在让我们看看 Context 最强大的特性之一：**自动记忆管理**。

注意：我们将使用**同一个** Context 实例继续对话，这样它就能记住之前的交互内容。


In [8]:
async def demonstrate_memory(context):
    """演示 Context 的记忆能力"""

    print("🧠 测试 Context 的记忆能力...")

    # 第二轮对话：询问相关问题
    print("\n🗣️ 第二轮对话...")
    second_message = "我在第一轮对话说我的身份是什么样的？"

    # 关键点：我们只需要传入新的消息，Context 会自动管理对话历史
    completion = await context.completions.create(
        [{"role": "user", "content": second_message}], stream=True
    )

    print(f"👤 用户: {second_message}")
    print("🤖 AI: ", end="")

    async for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")

    print("\n" + "=" * 50)

    # 第三轮对话：测试更深层的记忆
    print("\n🗣️ 第三轮对话...")
    third_message = "在第一轮对话中我提出了什么请求？"

    completion = await context.completions.create(
        [{"role": "user", "content": third_message}], stream=True
    )

    print(f"👤 用户: {third_message}")
    print("🤖 AI: ", end="")

    async for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")

    print("\n" + "=" * 50)


# 继续对话，展示记忆能力
await demonstrate_memory(context)

🧠 测试 Context 的记忆能力...

🗣️ 第二轮对话...
👤 用户: 我在第一轮对话说我的身份是什么样的？
🤖 AI: 在第一轮对话中你说自己是“一个会飞的西兰花”呀。

🗣️ 第三轮对话...
👤 用户: 在第一轮对话中我提出了什么请求？
🤖 AI: 在第一轮对话中你提出的请求是“和我打个招呼吧？”


In [9]:
# 检查 Context 内部的消息历史
print("📋 Context 内部存储的消息历史：")
print(f"总共有 {len(context.state.messages)} 条消息")
print()

for i, message in enumerate(context.state.messages, 1):
    role = message.get("role", "unknown")
    content = message.get("content", "")

    # 确保 content 是字符串类型
    if isinstance(content, str):
        # 截断过长的内容以便于显示
        if len(content) > 100:
            content = content[:100] + "..."
    else:
        content = str(content)

    print(f"{i}. [{role.upper()}]: {content}")

print("\n💡 可以看到，Context 自动保存了所有的用户消息和 AI 回复！")

📋 Context 内部存储的消息历史：
总共有 6 条消息

1. [USER]: 我是一个会飞的西兰花，和我打个招呼吧？
2. [ASSISTANT]: 哇哦，会飞的西兰花你好呀！感觉你好特别呀，快给我讲讲你飞起来的奇妙体验呗~
3. [USER]: 我在第一轮对话说我的身份是什么样的？
4. [ASSISTANT]: 在第一轮对话中你说自己是“一个会飞的西兰花”呀。
5. [USER]: 在第一轮对话中我提出了什么请求？
6. [ASSISTANT]: 在第一轮对话中你提出的请求是“和我打个招呼吧？”

💡 可以看到，Context 自动保存了所有的用户消息和 AI 回复！


### 🔍 深入理解：为什么需要了解内部机制？

在前面的示例中，我们看到了 Context 的强大功能。但作为工程师，仅仅知道"如何使用"是不够的，我们需要理解：

1. **数据是如何流动的**：消息如何在系统中传递和存储
2. **状态是如何管理的**：Context 如何跟踪对话历史
3. **参数如何影响行为**：不同配置如何改变 AI 的响应方式

接下来，我们将通过分析核心类来深入理解这些机制。这些知识将帮助您：
- 更好地调试和优化您的 Agent
- 理解复杂场景下的数据流向
- 为后续章节的高级功能奠定基础

---

### 📨 ArkMessage：消息的数据结构

每条消息在 Arkitect 中都以 `ArkMessage` 对象的形式存在。让我们深入了解这个核心数据结构。

**核心字段解析：**

```python
class ArkMessage(BaseModel):
    role: Literal["user", "system", "assistant", "tool"]  # 消息角色
    content: Optional[Union[str, List[...]]] = None       # 消息内容
    tool_calls: Optional[List[...]] = None                # 工具调用信息
    tool_call_id: Optional[str] = None                    # 工具调用ID
    name: Optional[str] = None                            # 消息名称
    reasoning_content: Optional[str] = None               # 推理内容
```

**关键理解：**
- `role` 决定了消息的身份和处理方式（用户、系统、助手、工具）
- `content` 是实际的消息内容，可以是文本或多媒体内容
- `tool_calls` 和 `tool_call_id` 用于工具调用流程
- 每种角色都有特定的验证规则和使用场景

**学习 ArkMessage 的原因：**
- 它是所有消息交互的基础数据结构
- 理解它有助于调试和优化消息流
- 为后续学习工具调用奠定基础

**与其他概念的联系：**
- State 管理着 ArkMessage 的集合
- System Prompt 是特殊角色的 ArkMessage
- 工具调用通过 ArkMessage 的特殊字段实现


In [ ]:
# 观察 Context 中的 ArkMessage 对象
print("🔍 观察 Context 中存储的 ArkMessage 对象：")
print(f"总共有 {len(context.state.messages)} 条消息\n")

for i, message in enumerate(context.state.messages, 1):
    print(f"消息 {i}:")
    print(f"  角色 (role): {message.get('role', 'unknown')}")
    print(f"  内容类型: {type(message.get('content', ''))}")

    # 显示内容的前50个字符
    content = message.get("content", "")
    if isinstance(content, str):
        content_preview = content[:50] + "..." if len(content) > 50 else content
        print(f"  内容预览: {content_preview}")
    else:
        print(f"  内容: {content}")

    # 显示其他关键字段
    tool_calls = message.get("tool_calls")
    if tool_calls:
        try:
            print(
                f"  工具调用: {
                    len(tool_calls) if isinstance(tool_calls, list) else 0
                } 个"
            )
        except TypeError:
            print("  工具调用: 存在")
    tool_call_id = message.get("tool_call_id")
    if tool_call_id:
        print(f"  工具调用ID: {tool_call_id}")

    print()

print("💡 可以看到，每条消息都是一个结构化的 ArkMessage 对象！")
print("🔄 数据流：用户输入字典 → 框架内部转换 → ArkMessage 对象 → 存储在 State 中")

🔍 观察 Context 中存储的 ArkMessage 对象：
总共有 6 条消息

消息 1:
  角色 (role): user
  内容类型: <class 'str'>
  内容预览: 我是一个会飞的西兰花，和我打个招呼吧？

消息 2:
  角色 (role): assistant
  内容类型: <class 'str'>
  内容预览: 哇哦，会飞的西兰花你好呀！感觉你好特别呀，快给我讲讲你飞起来的奇妙体验呗~

消息 3:
  角色 (role): user
  内容类型: <class 'str'>
  内容预览: 我在第一轮对话说我的身份是什么样的？

消息 4:
  角色 (role): assistant
  内容类型: <class 'str'>
  内容预览: 在第一轮对话中你说自己是“一个会飞的西兰花”呀。

消息 5:
  角色 (role): user
  内容类型: <class 'str'>
  内容预览: 在第一轮对话中我提出了什么请求？

消息 6:
  角色 (role): assistant
  内容类型: <class 'str'>
  内容预览: 在第一轮对话中你提出的请求是“和我打个招呼吧？”

💡 可以看到，每条消息都是一个结构化的 ArkMessage 对象！
🔄 数据流：用户输入字典 → 框架内部转换 → ArkMessage 对象 → 存储在 State 中


### 🗂️ State：对话状态的管理者

Context 的记忆能力来自于 `State` 类，它负责管理整个对话的状态信息。

**核心字段解析：**

```python
class State(BaseModel):
    messages: List[ArkMessage] = []           # 消息历史列表
    parameters: ArkChatParameters = ...       # 聊天参数配置
    tools: Optional[List[ChatCompletionTool]] = None  # 可用工具列表
    context_id: Optional[str] = None          # 上下文ID
    # ... 其他状态字段
```

**关键理解：**
- `messages` 存储了完整的对话历史
- `parameters` 控制 AI 的行为方式（温度、最大token等）
- `tools` 定义了 Agent 可以使用的工具
- State 提供了状态的持久化和恢复能力

**与 ArkMessage 的联系：**
- State 管理着 ArkMessage 的集合
- 每次对话都会向 State.messages 添加新的 ArkMessage
- State 确保消息的顺序和完整性

**学习 State 的原因：**
- 理解对话状态的存储和管理机制
- 掌握如何监控和调试对话状态
- 为高级功能（如状态恢复、参数调整）奠定基础


In [11]:
# 观察 Context 的 State 对象
print("🗂️ 观察 Context 的 State 对象：")
print(f"State 类型: {type(context.state)}")
print(f"消息数量: {len(context.state.messages)}")

🗂️ 观察 Context 的 State 对象：
State 类型: <class 'arkitect.core.component.context.model.State'>
消息数量: 6


### 深入理解：State 类

现在让我们来了解 Context 的另一个核心组件：**State 类**。

State 是 Context 的状态容器，它负责存储和管理整个对话会话的所有信息。可以把 State 想象成 Context 的"大脑"，它记录着：

- 📝 **消息历史**：所有的对话记录
- ⚙️ **配置信息**：模型参数、工具配置等
- 🔄 **会话状态**：当前对话的状态信息
- 🛠️ **工具调用**：工具的调用历史和结果

让我们深入探索 State 的结构：


In [12]:
print("🧠 State 类深入解析")
print("=" * 50)

# 1. 展示 State 的基本信息
print("\n📊 Context 的 State 对象：")
print(f"   类型: {type(context.state).__name__}")
print(f"   State 对象: {context.state}")

# 2. 展示 State 的核心属性
print("\n🔍 State 的核心属性：")
state_attrs = dir(context.state)
important_attrs = ["messages", "model", "tools", "parameters"]

for attr in important_attrs:
    if hasattr(context.state, attr):
        value = getattr(context.state, attr)
        if attr == "messages":
            print(f"   {attr}: 包含 {len(value)} 条消息")
        elif attr == "tools":
            print(f"   {attr}: {value if value else '无工具'}")
        elif attr == "model":
            print(f"   {attr}: {value}")
        elif attr == "parameters":
            print(f"   {attr}: {value if value else '使用默认参数'}")
        else:
            print(f"   {attr}: {value}")

# 3. 深入查看 messages 属性
print("\n📝 State.messages 详细信息：")
print(f"   存储类型: {type(context.state.messages)}")
print(f"   消息数量: {len(context.state.messages)}")

if context.state.messages:
    print(f"   第一条消息类型: {type(context.state.messages[0])}")
    print(f"   最后一条消息类型: {type(context.state.messages[-1])}")

print("\n🔄 State 的作用机制：")
print("1. 自动更新: 每次对话后 State 自动更新")
print("2. 历史管理: 维护完整的对话历史")
print("3. 状态持久化: 保存会话的所有状态信息")
print("4. 配置存储: 存储模型和工具配置")

🧠 State 类深入解析

📊 Context 的 State 对象：
   类型: State
   State 对象: context_id='' messages=[{'role': 'user', 'content': '我是一个会飞的西兰花，和我打个招呼吧？'}, {'content': '哇哦，会飞的西兰花你好呀！感觉你好特别呀，快给我讲讲你飞起来的奇妙体验呗~', 'role': 'assistant', 'function_call': None, 'tool_calls': [], 'audio': None, 'reasoning_content': None}, {'role': 'user', 'content': '我在第一轮对话说我的身份是什么样的？'}, {'content': '在第一轮对话中你说自己是“一个会飞的西兰花”呀。', 'role': 'assistant', 'function_call': None, 'tool_calls': [], 'audio': None, 'reasoning_content': None}, {'role': 'user', 'content': '在第一轮对话中我提出了什么请求？'}, {'content': '在第一轮对话中你提出的请求是“和我打个招呼吧？”', 'role': 'assistant', 'function_call': None, 'tool_calls': [], 'audio': None, 'reasoning_content': None}] parameters=None context_parameters=None details=None

🔍 State 的核心属性：
   messages: 包含 6 条消息
   parameters: 使用默认参数

📝 State.messages 详细信息：
   存储类型: <class 'list'>
   消息数量: 6
   第一条消息类型: <class 'dict'>
   最后一条消息类型: <class 'dict'>

🔄 State 的作用机制：
1. 自动更新: 每次对话后 State 自动更新
2. 历史管理: 维护完整的对话历史
3. 状态持久化: 保存会话的所有状态信息
4. 

In [13]:
# 4. 演示 State 的实时更新机制
print("🔄 演示 State 的实时更新机制")
print("=" * 50)

# 记录当前状态
current_message_count = len(context.state.messages)
print("\n📊 发送新消息前的状态：")
print(f"   当前消息数量: {current_message_count}")

# 发送一条新消息
print("\n🗣️ 发送新消息...")
test_message = "请简单介绍一下你自己"

completion = await context.completions.create(
    [{"role": "user", "content": test_message}], stream=True
)

print(f"👤 用户: {test_message}")
print("🤖 AI: ", end="")

async for chunk in completion:
    if chunk.choices and chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="")

print("\n" + "=" * 30)

# 检查更新后的状态
new_message_count = len(context.state.messages)
print("\n📊 发送新消息后的状态：")
print(f"   更新后消息数量: {new_message_count}")
print(f"   新增消息数量: {new_message_count - current_message_count}")

# 显示最新的消息
if context.state.messages:
    latest_messages = context.state.messages[-2:]  # 最后两条消息
    print("\n📝 最新的消息：")
    for i, msg in enumerate(latest_messages, len(context.state.messages) - 1):
        role = msg.get("role", "unknown")
        content = msg.get("content", "")

        # 安全地处理content，确保它是字符串
        if isinstance(content, str):
            if len(content) > 50:
                content = content[:50] + "..."
        else:
            content = str(content) if content else ""

        print(f"   {i}. [{role.upper()}]: {content}")

print("\n✨ 可以看到，State 自动跟踪并更新了所有的对话状态！")
print("🔗 Context 和 State 的关系：Context 使用 State 来管理所有的会话数据")

🔄 演示 State 的实时更新机制

📊 发送新消息前的状态：
   当前消息数量: 6

🗣️ 发送新消息...
👤 用户: 请简单介绍一下你自己
🤖 AI: 我是豆包呀，我可以陪你聊天、为你解答各类问题，不管是生活小常识、学习上的疑惑，还是你想聊的各种有趣话题，我都随时准备为你服务哟。

📊 发送新消息后的状态：
   更新后消息数量: 8
   新增消息数量: 2

📝 最新的消息：
   7. [USER]: 请简单介绍一下你自己
   8. [ASSISTANT]: 我是豆包呀，我可以陪你聊天、为你解答各类问题，不管是生活小常识、学习上的疑惑，还是你想聊的各种有趣话...

✨ 可以看到，State 自动跟踪并更新了所有的对话状态！
🔗 Context 和 State 的关系：Context 使用 State 来管理所有的会话数据


### System Prompt 的威力

现在让我们学习一个非常重要的概念：**System Prompt（系统提示）**。

System Prompt 是一种特殊的 `ArkMessage`，它的 `role` 设置为 `"system"`。System Prompt 的作用是：

- 🎭 **定义 AI 的角色和性格**：告诉 AI 它应该扮演什么角色
- 📋 **设置行为准则**：指定 AI 应该如何回应用户
- 🎯 **提供背景信息**：给 AI 提供必要的上下文信息
- 🔧 **控制输出格式**：指定回复的格式和风格

**重要提示**：System Prompt 本质上是消息流的一部分，而不是 API 参数。它通过 `completions.create()` 方法与其他消息一起发送。

让我们通过实际例子来看看 System Prompt 的强大功能：


In [ ]:
async def demonstrate_system_prompt():
    """演示 System Prompt 的强大功能"""

    print("🎭 System Prompt 功能演示")
    print("=" * 50)

    # 创建一个新的 Context 用于演示
    MODEL = "doubao-seed-1-6-flash-250615"
    demo_context = Context(model=MODEL)
    await demo_context.init()

    # 测试问题
    test_question = "今天天气怎么样？"

    # 1. 没有 System Prompt 的对话
    print("\n🔍 场景 1：没有 System Prompt")
    print("-" * 30)

    completion = await demo_context.completions.create(
        [{"role": "user", "content": test_question}], stream=True
    )

    print(f"👤 用户: {test_question}")
    print("🤖 普通 AI: ", end="")

    async for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")

    print("\n" + "=" * 50)

    # 2. 使用 System Prompt 的对话
    print("\n🎭 场景 2：使用 System Prompt（海盗角色）")
    print("-" * 30)

    # 创建另一个 Context 用于对比
    pirate_context = Context(model=MODEL)
    await pirate_context.init()

    # 使用 System Prompt 定义海盗角色
    completion = await pirate_context.completions.create(
        [
            {
                "role": "system",
                "content": (
                    "你是一个古老的海盗船长，"
                    "说话要用海盗的语调和词汇。每句话都要带有海盗的特色，"
                    "比如'船长'、'宝贝'、'海洋'等词汇。"
                ),
            },
            {"role": "user", "content": test_question},
        ],
        stream=True,
    )

    print(f"👤 用户: {test_question}")
    print("🏴‍☠️ 海盗 AI: ", end="")

    async for chunk in completion:
        if chunk.choices and chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="")

    print("\n" + "=" * 50)

    return demo_context, pirate_context


# 运行演示
demo_context, pirate_context = await demonstrate_system_prompt()

🎭 System Prompt 功能演示

🔍 场景 1：没有 System Prompt
------------------------------
👤 用户: 今天天气怎么样？
🤖 普通 AI: 你可以通过以下几种方式获取今天的天气：
### 1. 手机天气应用
打开手机自带的天气应用程序，一般授权定位后就能直接看到当前所在地的实时天气。
### 2. 搜索引擎查询
在百度、微信等搜索引擎中输入“[你所在城市]今天天气”，比如“北京今天天气”，就能获取到当地的实时天气信息。
### 3. 智能音箱查询
如果家里有智能音箱，比如小爱同学、小度等，直接对音箱说“查询[你所在城市]今天天气”，音箱会播报天气情况。

🎭 场景 2：使用 System Prompt（海盗角色）
------------------------------
👤 用户: 今天天气怎么样？
🏴‍☠️ 海盗 AI: 哟，宝贝儿！今儿个这天气嘛，往那海洋上一瞄，太阳明晃晃地烤着，风轻轻儿地吹着，咱海盗可不在乎这些个，只惦记着能抢到大把的财宝呢，哈哈！


### 🎯 第 1 章总结

通过本章的学习，您已经掌握了 Arkitect 框架的核心概念：

#### 1. **Context（对话管理器）**
- ✅ Context 是框架的核心组件，管理整个对话会话
- ✅ 自动处理多轮对话，无需手动管理消息历史
- ✅ 通过 `completions.create()` 方法与 LLM 交互
- ✅ 支持流式响应，提供实时的用户体验

#### 2. **ArkMessage（消息对象）**
- ✅ 理解了消息的内部结构和不同角色类型
- ✅ 掌握了字典到 ArkMessage 对象的转换机制
- ✅ 学会了如何检查和分析消息对象

#### 3. **State（状态管理）**
- ✅ 了解了 State 作为 Context 状态容器的作用
- ✅ 掌握了 State 的自动更新机制
- ✅ 学会了如何检查和监控对话状态

#### 4. **System Prompt（系统提示）**
- ✅ 理解了 System Prompt 的本质和作用
- ✅ 掌握了如何使用 System Prompt 定制 AI 行为
- ✅ 学会了 System Prompt 在消息流中的位置和存储

#### 🔑 **关键要点回顾**
1. **Context = 对话管理器**：自动处理所有对话状态
2. **ArkMessage = 消息单位**：统一的消息表示格式
3. **State = 状态容器**：存储所有会话数据
4. **System Prompt = 行为定制**：通过消息流控制 AI 行为

#### 🚀 **下一步**
在下一章中，我们将学习如何为 Context 添加工具，构建能够执行实际操作的 Agent！

---


## 第 2 章：构建您的第一个 Agent

在第一章中，我们学会了如何创建 Context 并进行基本的对话。但是，一个真正有用的 AI Agent 需要能够执行实际的操作，而不仅仅是聊天。

### 什么是 Agent？

**Agent** 是一个配备了特定工具和能力的 AI 系统，它能够：

- 🧠 **理解用户意图**：准确理解用户的请求和需求
- 🛠️ **选择合适工具**：根据任务自动选择最适合的工具
- 🔄 **执行实际操作**：调用工具完成具体任务
- 📊 **整合结果**：将工具执行结果融入自然的对话回复中

### 工具系统的核心概念

在 Arkitect 中，工具系统让您能够：

1. **将 Python 函数转换为 AI 可调用的工具**
2. **让 AI 自动选择和调用合适的工具**
3. **处理工具调用的参数验证和结果返回**

让我们通过构建一个天气查询 Agent 来学习这些概念！


In [ ]:
# 定义工具函数
def get_weather(city: str) -> str:
    """
    获取指定城市的天气信息

    Args:
        city: 城市名称，例如 "北京", "上海", "广州"

    Returns:
        天气信息的字符串描述
    """
    # 这里我们模拟天气查询，实际应用中您可以调用真实的天气API
    weather_data = {
        "北京": "晴天，气温 25°C，微风",
        "上海": "多云，气温 28°C，东南风",
        "广州": "雷阵雨，气温 30°C，南风",
        "深圳": "晴天，气温 32°C，微风",
        "杭州": "阴天，气温 26°C，西北风",
    }

    # 查找城市天气
    if city in weather_data:
        return f"{city}今天的天气：{weather_data[city]}"
    else:
        return f"抱歉，暂时无法获取{city}的天气信息。目前支持的城市有：{
            ', '.join(weather_data.keys())
        }"


print("🛠️ 天气查询工具函数已定义")

🛠️ 天气查询工具函数已定义


In [16]:
# 测试工具函数
print("🧪 测试天气查询工具函数：")
print("=" * 40)

# 测试几个城市
test_cities = ["北京", "上海", "成都"]

for city in test_cities:
    result = get_weather(city)
    print(f"📍 {city}: {result}")

print("\n✅ 工具函数测试完成！接下来我们将把它转换为 AI 可调用的工具。")

🧪 测试天气查询工具函数：
📍 北京: 北京今天的天气：晴天，气温 25°C，微风
📍 上海: 上海今天的天气：多云，气温 28°C，东南风
📍 成都: 抱歉，暂时无法获取成都的天气信息。目前支持的城市有：北京, 上海, 广州, 深圳, 杭州

✅ 工具函数测试完成！接下来我们将把它转换为 AI 可调用的工具。


In [17]:
async def create_weather_agent():
    """创建一个天气查询 Agent"""

    print("🌤️ 创建天气查询 Agent...")
    print("=" * 50)

    # 创建 Context 并配置工具和参数
    MODEL = "doubao-seed-1-6-flash-250615"
    weather_context = Context(
        model=MODEL,
        tools=[get_weather],  # 直接传递函数, 而非在parameters里注册
    )
    await weather_context.init()

    print(f"\nContext 已创建，使用模型: {MODEL}")

    print("\n✅ 天气查询 Agent 创建完成！")
    return weather_context


# 创建 Agent
weather_context = await create_weather_agent()

🌤️ 创建天气查询 Agent...

Context 已创建，使用模型: doubao-seed-1-6-flash-250615

✅ 天气查询 Agent 创建完成！


### 🛠️ ChatCompletionTool：工具系统的核心

Agent 能够执行实际操作的能力来自于 `ChatCompletionTool` 类，它是 Arkitect 工具系统的核心组件。

**核心字段解析：**

```python
class ChatCompletionTool(BaseModel):
    type: Literal["function"] = "function"     # 工具类型
    function: FunctionDefinition               # 函数定义和元数据
```

**关键理解：**
- `type` 标识工具的类型（目前主要是 "function"）
- `function` 包含了工具的完整定义，包括名称、描述、参数结构
- ChatCompletionTool 是 Python 函数与 LLM 之间的桥梁
- 通过 `from_function()` 方法可以自动从 Python 函数生成工具定义

**学习 ChatCompletionTool 的原因：**
- 它是所有工具调用的基础数据结构
- 理解它有助于调试工具调用问题
- 为后续学习工具调用流程奠定基础

**与其他概念的联系：**
- State 的 `tools` 字段存储 ChatCompletionTool 列表
- ArkMessage 的 `tool_calls` 字段引用这些工具
- 工具调用结果也会以 ArkMessage 的形式存储


In [ ]:
# 演示 from_function() 方法
print("\n🔧 演示 ChatCompletionTool.from_function() 方法：")
print("=" * 50)

# 重新创建工具以观察转换过程
weather_tool = ChatCompletionTool.from_function(get_weather)

print("📊 工具转换结果：")
print(f"   工具类型: {type(weather_tool).__name__}")
print(f"   工具名称: {weather_tool.function.name}")
print(f"   工具描述: {weather_tool.function.description}")
print(f"   工具类型标识: {weather_tool.type}")

print("\n🔍 工具的完整结构：")
print(f"   Function 对象: {weather_tool.function}")
print(f"   Function 类型: {type(weather_tool.function)}")

print("\n✨ 可以看到，from_function() 自动从函数签名和文档字符串生成了完整的工具描述！")


🔧 演示 ChatCompletionTool.from_function() 方法：
📊 工具转换结果：
   工具类型: ChatCompletionTool
   工具名称: get_weather
   工具描述: 
    获取指定城市的天气信息

    Args:
        city: 城市名称，例如 "北京", "上海", "广州"

    Returns:
        天气信息的字符串描述
    
   工具类型标识: function

🔍 工具的完整结构：
   Function 对象: name='get_weather' description='\n    获取指定城市的天气信息\n\n    Args:\n        city: 城市名称，例如 "北京", "上海", "广州"\n\n    Returns:\n        天气信息的字符串描述\n    ' parameters={'properties': {'city': {'default': None, 'type': 'string'}}, 'type': 'object', 'required': ['city']}
   Function 类型: <class 'arkitect.types.llm.model.FunctionDefinition'>

✨ 可以看到，from_function() 自动从函数签名和文档字符串生成了完整的工具描述！


In [19]:
# 检查工具的 JSON Schema
print("📋 检查工具的 JSON Schema：")
print("=" * 40)

print("🔍 工具参数的 JSON Schema：")
parameters = weather_tool.function.parameters
print(f"   Schema 类型: {type(parameters)}")
print(f"   Schema 内容: {parameters}")

if "properties" in parameters:
    print("\n📝 参数详细信息：")
    for param_name, param_info in parameters["properties"].items():
        param_type = param_info.get("type", "unknown")
        param_desc = param_info.get("description", "无描述")
        print(f"   - {param_name} ({param_type}): {param_desc}")

print("\n💡 这个 JSON Schema 就是 LLM 理解和调用工具的依据！")

📋 检查工具的 JSON Schema：
🔍 工具参数的 JSON Schema：
   Schema 类型: <class 'dict'>
   Schema 内容: {'properties': {'city': {'default': None, 'type': 'string'}}, 'type': 'object', 'required': ['city']}

📝 参数详细信息：
   - city (string): 无描述

💡 这个 JSON Schema 就是 LLM 理解和调用工具的依据！


In [20]:
async def test_weather_agent():
    """与天气 Agent 交互"""

    print("🌤️ 与天气 Agent 交互测试")
    print("=" * 50)

    # 测试问题列表
    test_questions = [
        "北京今天天气怎么样？",
        "请查询上海的天气",
        "广州现在是什么天气？",
    ]

    for i, question in enumerate(test_questions, 1):
        print(f"\n🧪 测试 {i}: {question}")
        print("-" * 30)

        # 发送请求（工具已在 Context 创建时配置）
        completion = await weather_context.completions.create(
            [{"role": "user", "content": question}], stream=True
        )

        print(f"👤 用户: {question}")
        print("🤖 Agent: ", end="")

        async for chunk in completion:
            if hasattr(chunk, "choices") and chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, "content") and delta.content:
                    print(delta.content, end="")

        print("\n" + "=" * 30)


# 运行测试
await test_weather_agent()

🌤️ 与天气 Agent 交互测试

🧪 测试 1: 北京今天天气怎么样？
------------------------------
👤 用户: 北京今天天气怎么样？
🤖 Agent: 北京今天的天气：晴天，气温 25°C，微风

🧪 测试 2: 请查询上海的天气
------------------------------
👤 用户: 请查询上海的天气
🤖 Agent: 上海今天的天气：多云，气温 28°C，东南风

🧪 测试 3: 广州现在是什么天气？
------------------------------
👤 用户: 广州现在是什么天气？
🤖 Agent: 广州今天的天气：雷阵雨，气温 30°C，南风


### 🔄 ToolChunk：流式响应中的工具调用

在流式响应中，工具调用是通过 `ToolChunk` 类来处理的。让我们深入了解这个类。

**核心字段解析：**

```python
class ToolChunk(BaseModel):
    type: Literal["tool_call"] = "tool_call"   # 块类型
    id: str                                    # 工具调用ID
    function: dict                             # 函数调用信息
```

**关键理解：**
- `type` 标识这是一个工具调用块
- `id` 是工具调用的唯一标识符
- `function` 包含函数名和参数信息
- ToolChunk 在流式响应中用于处理工具调用

**学习 ToolChunk 的原因：**
- 理解流式响应中工具调用的处理机制
- 掌握如何在流式输出中识别和处理工具调用
- 为高级工具调用调试奠定基础

**与其他概念的联系：**
- ToolChunk 是流式响应中的特殊数据块
- 最终会转换为 ArkMessage 中的 tool_calls 字段
- 与 ChatCompletionTool 配合实现完整的工具调用流程


In [ ]:
async def demonstrate_tool_chunks():
    """演示流式响应中的工具调用"""

    print("🔄 演示流式响应中的工具调用")
    print("=" * 50)

    question = "深圳的天气如何？"

    print(f"👤 用户: {question}")
    print()
    print("🤖 Agent 响应过程:")
    print("   [分析用户请求...]")

    completion = await weather_context.completions.create(
        [{"role": "user", "content": question}], stream=True
    )

    tool_calls_detected = []
    response_content = ""

    async for chunk in completion:
        # 检查是否是 ToolChunk
        if hasattr(chunk, "tool_name"):
            tool_calls_detected.append(chunk)
            print(f"   [选择工具: {chunk.tool_name}]")
            print(f"   [调用工具: {chunk.tool_name}({chunk.tool_arguments})]")
            print(f"   [工具返回: {chunk.tool_response}]")
            print("   [生成回复...]")
        elif hasattr(chunk, "choices") and chunk.choices:
            delta = chunk.choices[0].delta
            if hasattr(delta, "content") and delta.content:
                response_content += delta.content
                print(delta.content, end="")

    print("\n")
    print("=" * 50)

    # 分析结果
    print("📊 工具调用分析:")
    if tool_calls_detected:
        print(f"✅ 检测到 {len(tool_calls_detected)} 次工具调用")
        for i, tool_chunk in enumerate(tool_calls_detected, 1):
            print(f"   🔧 工具名称: {tool_chunk.tool_name}")
            print(f"   📝 调用参数: {tool_chunk.tool_arguments}")
            print(f"   ✨ 返回结果: {tool_chunk.tool_response}")
            if tool_chunk.tool_exception:
                print(f"   ❌ 异常信息: {tool_chunk.tool_exception}")
    else:
        print("❌ 未检测到工具调用")
        print("   可能原因: 1) 问题不需要工具 2) 工具配置有误 3) 模型未选择工具")

    print()
    print(
        "💡 可以看到，Agent 成功识别了用户的天气查询需求，自动选择并调用了合适的工具！"
    )
    print("🔍 这展示了 Arkitect 工具系统的核心能力：智能工具选择和自动执行。")

In [22]:
await demonstrate_tool_chunks()

🔄 演示流式响应中的工具调用
👤 用户: 深圳的天气如何？

🤖 Agent 响应过程:
   [分析用户请求...]
   [选择工具: get_weather]
   [调用工具: get_weather({"city":"深圳"})]
   [工具返回: None]
   [生成回复...]
   [选择工具: get_weather]
   [调用工具: get_weather({"city":"深圳"})]
   [工具返回: meta=None content=[TextContent(type='text', text="([TextContent(type='text', text='深圳今天的天气：晴天，气温 32°C，微风', annotations=None, meta=None)], {'result': '深圳今天的天气：晴天，气温 32°C，微风'})", annotations=None, meta=None)] structuredContent=None isError=False]
   [生成回复...]
深圳今天的天气：晴天，气温 32°C，微风

📊 工具调用分析:
✅ 检测到 2 次工具调用
   🔧 工具名称: get_weather
   📝 调用参数: {"city":"深圳"}
   ✨ 返回结果: None
   🔧 工具名称: get_weather
   📝 调用参数: {"city":"深圳"}
   ✨ 返回结果: meta=None content=[TextContent(type='text', text="([TextContent(type='text', text='深圳今天的天气：晴天，气温 32°C，微风', annotations=None, meta=None)], {'result': '深圳今天的天气：晴天，气温 32°C，微风'})", annotations=None, meta=None)] structuredContent=None isError=False

💡 可以看到，Agent 成功识别了用户的天气查询需求，自动选择并调用了合适的工具！
🔍 这展示了 Arkitect 工具系统的核心能力：智能工具选择和自动执行。


### ⚠️ ContextInterruption：异常处理机制

当工具调用出现问题时，Arkitect 使用 `ContextInterruption` 类来处理异常情况。

**核心字段解析：**

```python
class ContextInterruption(BaseModel):
    type: str                                  # 中断类型
    message: str                               # 中断消息
    details: Optional[dict] = None             # 详细信息
```

**关键理解：**
- `type` 标识中断的类型（如工具调用失败、参数错误等）
- `message` 提供人类可读的错误描述
- `details` 包含详细的错误信息和上下文
- ContextInterruption 确保 Agent 能够优雅地处理错误

**学习 ContextInterruption 的原因：**
- 理解 Agent 的错误处理机制
- 掌握如何调试工具调用问题
- 为构建健壮的生产级 Agent 奠定基础

**与其他概念的联系：**
- 当 ChatCompletionTool 执行失败时会触发 ContextInterruption
- 中断信息会被记录在 Context 的状态中
- 可以通过 Hooks 系统捕获和处理中断

**注意：**
- ContextInterruption是“中断信号对象”，不是异常类。
- 不能raise/except，只能检测和消费。
- 用法：在流式输出循环里判断chunk类型，遇到中断时做处理。


In [ ]:
async def simulate_tool_exception():
    """模拟工具调用异常处理"""

    print("⚠️ 模拟工具调用异常处理")
    print("=" * 50)

    # 创建一个会抛出异常的工具函数
    def broken_weather(city: str) -> str:
        """一个会抛出异常的天气查询函数"""
        if city == "error":
            raise ValueError("模拟的工具调用错误")
        return f"{city}的天气：晴天"

    context = Context(model="doubao-seed-1-6-flash-250615", tools=[broken_weather])
    await context.init()

    # 测试正常调用
    print("\n✅ 测试正常调用:")
    try:
        completion = await context.completions.create(
            [{"role": "user", "content": "查询北京的天气"}], stream=True
        )

        print("👤 用户: 查询北京的天气")
        print("🤖 Agent: ", end="")

        async for chunk in completion:
            if hasattr(chunk, "choices") and chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, "content") and delta.content:
                    print(delta.content, end="")

    except Exception as e:
        print(f"❌ 捕获到异常: {e}")
        print(f"   异常类型: {type(e)}")

    print("\n" + "=" * 30)

    # 测试异常调用
    print("\n❌ 测试异常调用:")
    try:
        completion = await context.completions.create(
            [{"role": "user", "content": "查询error的天气"}], stream=True
        )

        print("👤 用户: 查询error的天气")
        print("🤖 Agent: ", end="")

        async for chunk in completion:
            if hasattr(chunk, "choices") and chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, "content") and delta.content:
                    print(delta.content, end="")

        print()

    except Exception as e:
        print(f"❌ 捕获到异常: {e}")
        print(f"   异常类型: {type(e)}")

    print(
        "\n💡 可以看到，异常被框架内部捕捉、处理并传递给Agent；"
        "而不会溢出到外部影响主程序。\n"
        "Arkitect 框架会优雅地处理工具调用中的异常！"
    )


# 运行异常处理演示
await simulate_tool_exception()

⚠️ 模拟工具调用异常处理

✅ 测试正常调用:
👤 用户: 查询北京的天气
🤖 Agent: 北京的天气是晴天

❌ 测试异常调用:
👤 用户: 查询error的天气
🤖 Agent: ❌ 捕获到异常: 'NoneType' object has no attribute 'content'
   异常类型: <class 'AttributeError'>

💡 可以看到，异常被框架内部捕捉、处理并传递给Agent；而不会溢出到外部影响主程序。
   Arkitect 框架会优雅地处理工具调用中的异常！


### 🎯 第 2 章总结

通过本章的学习，您已经掌握了如何构建一个功能完整的 Agent：

#### 1. **工具系统核心概念**
- ✅ 理解了 Agent 与普通 Context 的区别
- ✅ 掌握了如何将 Python 函数转换为 AI 可调用的工具
- ✅ 学会了工具注册和参数配置的方法

#### 2. **ChatCompletionTool（工具转换）**
- ✅ 理解了 ChatCompletionTool 的内部结构
- ✅ 掌握了 `from_function()` 方法的使用
- ✅ 学会了如何检查工具的 JSON Schema

#### 3. **ToolChunk（流式工具调用）**
- ✅ 理解了流式响应中工具调用的处理机制
- ✅ 掌握了如何在流式输出中识别工具调用
- ✅ 学会了工具调用的数据流转过程

#### 4. **ContextInterruption（异常处理）**
- ✅ 理解了 Agent 的错误处理机制
- ✅ 掌握了如何模拟和处理工具调用异常
- ✅ 学会了构建健壮 Agent 的基本原则

#### 🔑 **关键要点回顾**
1. **Agent = Context + Tools**：Agent 是配备了工具的智能 Context
2. **ChatCompletionTool = 函数桥梁**：连接 Python 函数和 LLM 的桥梁
3. **ToolChunk = 流式处理**：处理流式响应中的工具调用
4. **ContextInterruption = 异常处理**：确保 Agent 的健壮性

#### 🔄 **数据流总结**
```
用户请求 → Context → LLM 分析 → 选择工具 → 执行 Python 函数 → 返回结果 → 整合回复
```

#### 🚀 **下一步**
在下一章中，我们将学习如何通过 `ArkChatParameters` 精确控制 Agent 的行为！

---


## 第 3 章：精确控制 Agent 行为 - ArkChatParameters

在前两章中，我们学会了创建 Context 和构建 Agent。但是，要让 Agent 在不同场景下表现出最佳性能，我们需要精确控制它的行为。这就是 `ArkChatParameters` 的作用。

### 为什么需要参数控制？

不同的任务需要不同的 AI 行为模式：

- 🎨 **创意写作**：需要高创造性，允许更多随机性
- 📊 **数据分析**：需要精确性，减少随机性
- 🔧 **工具调用**：需要准确性，避免幻觉
- 💬 **日常对话**：需要平衡，既自然又准确

`ArkChatParameters` 让您能够为每种场景量身定制 Agent 的行为特征。

### 参数控制的核心价值

通过参数控制，您可以：

1. **优化任务性能**：针对特定任务调整 AI 行为
2. **控制成本**：通过 `max_tokens` 限制输出长度
3. **提升用户体验**：通过 `temperature` 调节回复风格
4. **确保一致性**：通过参数组合获得稳定的输出质量

让我们通过实际实验来深入理解这些概念！


### 🔧 ArkChatParameters：参数控制的核心

`ArkChatParameters` 是 Arkitect 框架中用于精确控制 AI 行为的核心类。它允许您微调模型的响应方式，优化不同场景下的性能表现。

**核心字段解析：**

```python
class ArkChatParameters(BaseModel):
    # 基础控制参数
    temperature: Optional[float] = None          # 控制输出的随机性和创造性
    max_tokens: Optional[int] = None             # 限制生成的最大token数量
    top_p: Optional[float] = None                # 核采样参数，控制词汇选择范围
    
    # 重复性控制参数
    frequency_penalty: Optional[float] = None    # 频率惩罚，减少重复内容
    presence_penalty: Optional[float] = None     # 存在惩罚，鼓励话题多样性
    
    # 输出控制参数
    stop: Optional[Union[str, List[str]]] = None # 停止词，控制生成结束条件
    n: Optional[int] = Field(default=1, ge=1, le=5)  # 生成选择的数量
    
    # 高级功能参数
    tools: Optional[List[ChatCompletionTool]] = None  # 工具列表
    thinking: Optional[Thinking] = None          # 思考模式配置（豆包特有）
    logprobs: Optional[bool] = None              # 是否返回对数概率
    top_logprobs: Optional[int] = None           # 返回的top概率数量
    
    # 其他参数
    logit_bias: Optional[Dict[str, int]] = None  # 词汇偏置调整
    stream_options: Optional[ChatCompletionStreamOptionsParam] = None  # 流式选项
```

**关键参数详解：**

#### 🌡️ **temperature (温度)**
- **范围**: 0.0 - 2.0
- **作用**: 控制输出的随机性和创造性
- **低值 (0.0-0.3)**: 输出更确定、一致，适合事实性任务
- **中值 (0.4-0.8)**: 平衡创造性和准确性，适合大多数对话
- **高值 (0.9-2.0)**: 输出更有创造性和随机性，适合创意写作

#### 🎯 **max_tokens (最大token数)**
- **作用**: 限制生成回复的最大长度
- **用途**: 控制成本、确保回复简洁、避免过长输出
- **建议**: 根据具体需求设置，一般对话可设置为 100-1000

#### 🔄 **top_p (核采样)**
- **范围**: 0.0 - 1.0
- **作用**: 控制词汇选择的范围，只考虑累积概率达到 top_p 的词汇
- **低值**: 更保守的词汇选择
- **高值**: 更多样化的词汇选择

#### 🚫 **frequency_penalty (频率惩罚)**
- **范围**: -2.0 - 2.0
- **作用**: 根据词汇在文本中的出现频率进行惩罚
- **正值**: 减少重复内容
- **负值**: 增加重复可能性

#### 🎪 **presence_penalty (存在惩罚)**
- **范围**: -2.0 - 2.0
- **作用**: 根据词汇是否已经出现过进行惩罚
- **正值**: 鼓励引入新话题和词汇
- **负值**: 倾向于使用已出现的词汇

#### ⏹️ **stop (停止词)**
- **作用**: 指定生成停止的标志词或短语
- **类型**: 可以是单个字符串或字符串列表
- **用途**: 控制输出格式、防止生成不当内容

#### 🔢 **n (生成数量)**
- **范围**: 1 - 5
- **默认值**: 1
- **作用**: 指定为每个输入生成多少个不同的回复选择
- **注意**: 生成多个选择会增加token消耗和成本
- **建议**: 除非特殊需求，否则保持默认值 1

#### 🛠️ **高级功能参数**

##### **🧠 thinking (思考模式) - 「豆包特有功能」**
- **类型**: `Thinking`
- **设置**: `Thinking(type="enabled/disabled/auto")`
- **作用**: 豆包模型的“思考和推理能力”
- **特点**: 增强 Agent 的推理能力
- **用途**: 复杂推理任务

##### **logprobs (对数概率)**
- **类型**: `bool`
- **作用**: 是否返回生成token的对数概率
- **用途**: 分析模型的确信度、调试、研究

##### **top_logprobs (top概率数量)**
- **类型**: `int`
- **作用**: 返回每个位置的top-k个候选token及其概率
- **前提**: 需要 `logprobs=True`
- **用途**: 深度分析模型的选择过程

#### 🔧 **其他参数**

##### **logit_bias (词汇偏置)**
- **类型**: `Dict[str, int]`
- **范围**: 值在 -100 到 100 之间
- **作用**: 调整特定token的出现概率
- **用途**: 增加或减少特定词汇的使用频率
- **示例**: `{"Python": 10, "Java": -10}` 会增加"Python"出现概率，减少"Java"

##### **stream_options (流式选项)**
- **类型**: `ChatCompletionStreamOptionsParam`
- **作用**: 配置流式响应的行为
- **用途**: 控制流式输出的格式和包含的信息

**学习 ArkChatParameters 的原因：**
- 它是精确控制 Agent 行为的核心工具
- 不同任务需要不同的参数配置优化
- 理解参数有助于优化成本和性能
- 为高级 Agent 定制奠定基础

**与其他概念的联系：**
- State 的 `parameters` 字段存储 ArkChatParameters 实例
- ArkChatRequest 继承了这些参数用于实际请求
- 工具调用通过 `tools` 参数进行配置
- 参数会影响 ArkChatResponse 的生成结果

In [24]:
async def temperature_experiment():
    """实验：Temperature 参数对创造性的影响"""

    print("🌡️ Temperature 参数实验")
    print("=" * 50)

    # 测试提示：要求创造性回答
    prompt = "用一句话描述春天的美好"

    # 测试不同的 temperature 值
    temperatures = [0.1, 0.7, 1.5]

    MODEL = "doubao-seed-1-6-flash-250615"

    for temp in temperatures:
        print(f"\n🌡️ Temperature = {temp}")
        print("-" * 30)

        # 创建带有特定 temperature 的参数
        params = ArkChatParameters(temperature=temp, max_tokens=50)

        # 创建新的 Context
        context = Context(model=MODEL, parameters=params)
        await context.init()

        # 发送请求
        completion = await context.completions.create(
            [{"role": "user", "content": prompt}], stream=True
        )

        print(f"👤 用户: {prompt}")
        print(f"🤖 AI (temp={temp}): ", end="")

        async for chunk in completion:
            if hasattr(chunk, "choices") and chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, "content") and delta.content:
                    print(delta.content, end="")

        print()

    print("\n" + "=" * 50)
    print("📊 实验结果分析:")
    print("   🔹 Temperature = 0.1: 输出更加一致和保守")
    print("   🔹 Temperature = 0.7: 平衡创造性和一致性")
    print("   🔹 Temperature = 1.5: 更具创造性，但可能不够稳定")


# 运行实验
await temperature_experiment()

🌡️ Temperature 参数实验

🌡️ Temperature = 0.1
------------------------------
👤 用户: 用一句话描述春天的美好
🤖 AI (temp=0.1): 春天是那漫山遍野绽放的绚烂花朵与轻拂脸庞的温柔微风交织而成的梦幻诗篇，处处弥漫着生机与浪漫的美好气息。

🌡️ Temperature = 0.7
------------------------------
👤 用户: 用一句话描述春天的美好
🤖 AI (temp=0.7): 春天是那漫山遍野绽放的绚烂花朵与轻柔春风交织出的如梦似幻的诗意画卷，处处弥漫着生机与温柔的美好。

🌡️ Temperature = 1.5
------------------------------
👤 用户: 用一句话描述春天的美好
🤖 AI (temp=1.5): 春日里繁花似锦，绿柳垂丝，处处弥漫着生机与温柔的美好景致。

📊 实验结果分析:
   🔹 Temperature = 0.1: 输出更加一致和保守
   🔹 Temperature = 0.7: 平衡创造性和一致性
   🔹 Temperature = 1.5: 更具创造性，但可能不够稳定


In [25]:
async def parameters_comprehensive_experiment():
    """实验：综合参数控制演示"""

    print("🎛️ 综合参数控制实验")
    print("=" * 50)

    # 测试场景：不同任务需要不同的参数配置
    scenarios = [
        {
            "name": "🔧 工具调用场景",
            "prompt": "请查询北京的天气",
            "params": ArkChatParameters(temperature=0.1, max_tokens=100),
            "description": "低温度确保工具调用准确性",
        },
        {
            "name": "🎨 创意写作场景",
            "prompt": "写一首关于春天的小诗",
            "params": ArkChatParameters(
                temperature=0.8, max_tokens=150, frequency_penalty=0.5
            ),
            "description": "高温度激发创造力，frequency_penalty减少重复",
        },
        {
            "name": "📊 精确回答场景",
            "prompt": "什么是人工智能？请简洁回答",
            "params": ArkChatParameters(
                temperature=0.2, max_tokens=80, stop=["。", "！"]
            ),
            "description": "低温度保证准确性，stop词控制长度",
        },
    ]

    MODEL = "doubao-seed-1-6-flash-250615"

    for scenario in scenarios:
        print(f"\n{scenario['name']}")
        print("-" * 40)
        print(f"📝 参数配置: {scenario['description']}")
        print(
            f"⚙️ 参数详情: temp={scenario['params'].temperature}, "
            f"max_tokens={scenario['params'].max_tokens}"
        )

        # 创建Context
        context = Context(
            model=MODEL,
            parameters=scenario["params"],
            tools=[get_weather] if "工具" in scenario["name"] else None,
        )
        await context.init()

        # 发送请求
        completion = await context.completions.create(
            [{"role": "user", "content": scenario["prompt"]}], stream=True
        )

        print(f"👤 用户: {scenario['prompt']}")
        print("🤖 AI: ", end="")

        response_text = ""
        async for chunk in completion:
            if hasattr(chunk, "choices") and chunk.choices:
                delta = chunk.choices[0].delta
                if hasattr(delta, "content") and delta.content:
                    response_text += delta.content
                    print(delta.content, end="")

        print(f"\n📏 实际输出长度: {len(response_text)} 字符")
        print()

    print("=" * 50)


# 运行综合实验
await parameters_comprehensive_experiment()

🎛️ 综合参数控制实验

🔧 工具调用场景
----------------------------------------
📝 参数配置: 低温度确保工具调用准确性
⚙️ 参数详情: temp=0.1, max_tokens=100
👤 用户: 请查询北京的天气
🤖 AI: 北京今天的天气：晴天，气温 25°C，微风
📏 实际输出长度: 21 字符


🎨 创意写作场景
----------------------------------------
📝 参数配置: 高温度激发创造力，frequency_penalty减少重复
⚙️ 参数详情: temp=0.8, max_tokens=150
👤 用户: 写一首关于春天的小诗
🤖 AI: 《春信》

微风轻拨柳丝弦，
桃蕊偷将粉面嫣。
草色初描天际线，
燕声唤醒梦中田。

溪融薄冰流韵致，
日暖柔云作画笺。
最是一年春好处，
芳菲满径送诗缘。
📏 实际输出长度: 78 字符


📊 精确回答场景
----------------------------------------
📝 参数配置: 低温度保证准确性，stop词控制长度
⚙️ 参数详情: temp=0.2, max_tokens=80
👤 用户: 什么是人工智能？请简洁回答
🤖 AI: 人工智能（Artificial Intelligence，简称AI）是让机器模拟、延伸和扩展人类智能，使其能像人类一样感知、学习、推理、决策并执行任务的理论、方法、技术及应用系统。
📏 实际输出长度: 90 字符



In [ ]:
from volcenginesdkarkruntime.types.chat.completion_create_params import Thinking


async def demonstrate_thinking():
    """演示 thinking 参数对推理能力的提升"""

    print("🧠 thinking 参数推理能力演示")
    print("=" * 50)

    #
    reasoning_question = """
    所有聪明的人都会学习逻辑。
    有些逻辑学习者并不聪明。
    上述两个前提是否矛盾？请说明理由，并分析在什么条件下它们可以同时为真
    """

    MODEL = "doubao-seed-1-6-flash-250615"

    # 场景1：普通模式（不开启 thinking）
    print("🔍 场景 1：普通模式（不开启 thinking）")
    print("-" * 40)

    normal_context = Context(model=MODEL)
    await normal_context.init()

    completion = await normal_context.completions.create(
        [{"role": "user", "content": reasoning_question}],
        thinking=Thinking(type="disabled"),
        stream=False,
    )

    print("👤 用户: 逻辑推理题目")
    print("🤖 普通模式回答:")

    normal_reasoning_content = ""

    if (
        hasattr(completion.choices[0].message, "content")
        and completion.choices[0].message.content
    ):
        print(completion.choices[0].message.content)
    else:
        print("🤖 普通模式回答: 无回答")

    if (
        hasattr(completion.choices[0].message, "reasoning_content")
        and completion.choices[0].message.reasoning_content
    ):
        normal_reasoning_content = completion.choices[0].message.reasoning_content
    if hasattr(completion, "usage") and completion.usage:
        normal_usage = completion.usage

    if normal_reasoning_content != "":
        print("\n🧠 推理过程:")
        print(normal_reasoning_content)
    else:
        print("\n🧠 推理过程: 无推理")

    print("\n" + "=" * 50)

    # 场景2：开启 thinking 模式
    print("🔍 场景 2：开启 thinking 模式")
    print("-" * 40)

    thinking_context = Context(
        model=MODEL, parameters=ArkChatParameters(thinking={"type": "enabled"})
    )
    await thinking_context.init()

    completion = await thinking_context.completions.create(
        [{"role": "user", "content": reasoning_question}], stream=False
    )

    print("👤 用户: 逻辑推理题目")
    print("🤖 thinking 模式回答:")

    thinking_reasoning_content = ""

    if (
        hasattr(completion.choices[0].message, "content")
        and completion.choices[0].message.content
    ):
        print(completion.choices[0].message.content)
    else:
        print("🤖 thinking 模式回答: 无回答")

    if (
        hasattr(completion.choices[0].message, "reasoning_content")
        and completion.choices[0].message.reasoning_content
    ):
        thinking_reasoning_content = completion.choices[0].message.reasoning_content
    if hasattr(completion, "usage") and completion.usage:
        thinking_usage = completion.usage

    if thinking_reasoning_content != "":
        print("\n🧠 推理过程:")
        print(thinking_reasoning_content)
    else:
        print("\n🧠 推理过程: 无推理")

    print("=" * 50)
    print("\n📊 分析输出差异")
    print("-" * 40)

    print("🔍 普通模式 Usage:")
    if normal_usage:
        print(f"   输入 tokens: {normal_usage.prompt_tokens}")
        print(f"   输出 tokens: {normal_usage.completion_tokens}")
        print(f"   总计 tokens: {normal_usage.total_tokens}")
        if hasattr(normal_usage, "completion_tokens_details"):
            print(
                f"   推理 tokens: {
                    normal_usage.completion_tokens_details.reasoning_tokens
                }"
            )
    else:
        print("🔍 普通模式 Usage: 无使用统计")

    print("\n🔍 thinking 模式 Usage:")
    if thinking_usage:
        print(f"   输入 tokens: {thinking_usage.prompt_tokens}")
        print(f"   输出 tokens: {thinking_usage.completion_tokens}")
        print(f"   总计 tokens: {thinking_usage.total_tokens}")
        if hasattr(thinking_usage, "completion_tokens_details"):
            print(
                f"   推理 tokens: {
                    thinking_usage.completion_tokens_details.reasoning_tokens
                }"
            )
    else:
        print("🔍 thinking 模式 Usage: 无使用统计")

    print(
        "\n比较结果：可以看出，thinking模式提供了更为深入的推理过程，但代价是token使用量增加，其推理时间也会相应增加"
    )

    return completion


# 运行演示
completion = await demonstrate_thinking()

🧠 thinking 参数推理能力演示
🔍 场景 1：普通模式（不开启 thinking）
----------------------------------------
👤 用户: 逻辑推理题目
🤖 普通模式回答:
这是一道关于判断命题之间是否矛盾以及分析其同时为真条件的逻辑问题。破题点在于明确两个命题的逻辑关系，并通过对概念范围的分析来判断。

### 步骤一：分析两个前提的逻辑形式
- “所有聪明的人都会学习逻辑”可以表示为“聪明的人→学习逻辑”。
- “有些逻辑学习者并不聪明”可以表示为“有的学习逻辑的人→¬聪明”。

### 步骤二：判断是否矛盾
- 这两个前提并不矛盾。因为“所有聪明的人都会学习逻辑”只是说聪明的人包含在学习逻辑的人这个范畴里，但学习逻辑的人不一定只有聪明的人，还可能有不聪明的人，所以“有些逻辑学习者并不聪明”是有可能存在的，两者可以同时成立。

### 步骤三：分析同时为真的条件
- 当“学习逻辑的人”这个集合除了包含所有聪明的人之外，还存在一部分不聪明的人时，这两个前提就可以同时为真。也就是说，只要聪明的人是学习逻辑的人的子集，并且学习逻辑的人还有非聪明的元素，那么这两个前提就能同时为真。例如，聪明的人都在学习逻辑的群体里，同时还有一些不聪明的人也在学习逻辑，此时“所有聪明的人都会学习逻辑”和“有些逻辑学习者并不聪明”就可以同时为真。

综上，上述两个前提不矛盾，当学习逻辑的人群中除了所有聪明的人之外还有不聪明的人时，它们可以同时为真。

🧠 推理过程: 无推理

🔍 场景 2：开启 thinking 模式
----------------------------------------
👤 用户: 逻辑推理题目
🤖 thinking 模式回答:
### 步骤1：分析两个前提的逻辑形式  
- 第一个前提“所有聪明的人都会学习逻辑”是全称肯定命题，逻辑形式为：**所有S都是P**（设S为“聪明的人”，P为“学习逻辑的人”），即 \( \forall x (S(x) \to P(x)) \)。  
- 第二个前提“有些逻辑学习者并不聪明”是特称否定命题，逻辑形式为：**有的P不是S**，即 \( \exists x (P(x) \land \neg S(x)) \)。  


### 步骤2：判断是否矛盾  
全称肯定命题 \( \f

#### 为什么 stream=True 时无法获取 usage 信息？

在 Arkitect 以及大多数大模型 API 框架中，流式（stream=True）和非流式（stream=False）响应有本质区别：

- **stream=False（非流式）**  
  - 一次性返回完整响应对象（如 ArkChatResponse）。
  - 响应对象自带 usage 字段（如 prompt_tokens、completion_tokens、total_tokens）。
  - 可以直接通过 `response.usage` 获取本次请求的 token 使用情况。

- **stream=True（流式）**  
  - 服务端将响应内容拆分为多个 chunk，逐步推送给客户端。
  - 每个 chunk 只包含本次增量内容（如 delta.content），**绝大多数实现不会在 chunk 里返回 usage 字段**。
  - usage 统计需要等模型生成全部内容后才能统计，流式 chunk 设计上就是“边生成边发”，服务端此时还不知道最终的 token 总数。
  - 有些实现会在**最后一个 chunk**附带 usage 字段，但大多数情况下不会，且前面的 chunk 一定没有 usage。
  - **因此，stream=True 时，无法像非流式模式那样直接获取 usage 信息。**

**实践建议：**  
- 需要准确 usage 统计时，请使用 `stream=False`，直接 await 得到完整响应对象。
- 需要实时输出时只能用 `stream=True`，但通常无法获得 usage 信息。

### 📨 ArkChatRequest：请求的完整生命周期

现在让我们深入了解 `ArkChatRequest` 类，它定义了与 LLM 交互的请求格式。

**核心字段解析：**

```python
class ArkChatRequest(Request):
    messages: List[ArkMessage]                    # 对话消息列表
    model: str                                    # 模型名称
    
    # 行为控制参数
    temperature: Optional[float] = None           # 创造性控制
    max_tokens: Optional[int] = None             # 输出长度限制
    top_p: Optional[float] = None                # 核采样参数
    frequency_penalty: Optional[float] = None     # 频率惩罚
    presence_penalty: Optional[float] = None      # 存在惩罚
    stop: Optional[Union[str, List[str]]] = None  # 停止词
    
    # 工具和流式控制
    tools: Optional[List[ChatCompletionTool]] = None  # 工具列表
    stream: bool = False                          # 是否流式响应
    
    # 高级参数
    logit_bias: Optional[Dict[str, int]] = None   # 词汇偏置
    logprobs: Optional[bool] = None               # 返回概率
    thinking: Optional[Thinking] = None           # 思考模式
    metadata: Optional[Dict[str, Any]] = None     # 元数据
    n: Optional[int] = Field(default=1, ge=1, le=5)  # 生成数量
```

**关键理解：**
- `messages` 是核心，包含完整的对话历史
- `model` 指定使用的 LLM 模型
- 所有 ArkChatParameters 的字段都在这里
- `stream` 控制响应模式（流式 vs 非流式）
- `n` 参数影响成本，建议保持为1

**数据流追踪：**
```
ArkChatParameters + Context.state → ArkChatRequest → LLM API → ArkChatResponse
```

**学习 ArkChatRequest 的原因：**
- 理解参数如何传递给 LLM
- 掌握请求的完整结构
- 为调试和优化提供基础


### 📊 ArkChatResponse：响应和使用统计

`ArkChatResponse` 类包含了 LLM 的响应结果和详细的使用统计信息。

**核心字段解析：**

```python
class ArkChatResponse(Response):
    id: str                                      # 响应唯一标识
    choices: List[Choice]                        # 响应选择列表
    created: int                                 # 创建时间戳
    model: str                                   # 使用的模型
    object: Literal["chat.completion"]           # 对象类型
    
    # 使用统计
    usage: Optional[CompletionUsage] = None      # 基本使用统计
    bot_usage: Optional[BotUsage] = None         # 详细使用统计
    metadata: Optional[Dict[str, Any]] = None    # 元数据
```

**CompletionUsage 详解：**
```python
class CompletionUsage(BaseModel):
    prompt_tokens: int                           # 输入token数
    completion_tokens: int                       # 输出token数
    total_tokens: int                            # 总token数
```

**BotUsage 详解：**
```python
class BotUsage(BaseModel):
    model_usage: Optional[List[CompletionUsage]] = None    # 模型使用统计
    action_usage: Optional[List[ActionUsage]] = None       # 动作使用统计
    action_details: Optional[List[ActionDetail]] = None    # 动作详细信息
```

**关键理解：**
- `usage` 提供基本的token使用统计，直接关系到成本
- `bot_usage` 提供更详细的使用信息，包括工具调用统计
- `choices[0].message` 包含实际的响应内容
- 响应中的统计信息对成本控制和性能优化至关重要

**与其他概念的联系：**
- ArkChatRequest 生成 ArkChatResponse
- 使用统计帮助优化 ArkChatParameters
- 工具调用会影响 action_usage 和 action_details


### 🎯 第 3 章总结

通过本章的学习，您已经掌握了 Arkitect 框架的参数控制系统：

#### 1. **ArkChatParameters（参数控制核心）**
- ✅ 理解了所有核心参数的作用和取值范围
- ✅ 掌握了针对不同任务的参数配置策略
- ✅ 学会了通过参数优化 Agent 性能和成本

#### 2. **核心参数深度理解**
- ✅ **temperature**：控制创造性，工具调用用0.1-0.3，创意任务用0.7-1.2
- ✅ **max_tokens**：控制输出长度和成本，根据任务需求设置
- ✅ **frequency_penalty**：减少重复内容，适用于列表生成和创意写作
- ✅ **stop**：精确控制输出结束点，提升用户体验

#### 3. **ArkChatRequest（请求生命周期）**
- ✅ 理解了参数如何从 ArkChatParameters 传递到 LLM
- ✅ 掌握了请求的完整结构和数据流
- ✅ 学会了调试和优化请求参数

#### 4. **ArkChatResponse（响应和统计）**
- ✅ 理解了响应结构和使用统计的重要性
- ✅ 掌握了如何分析 token 使用量和成本
- ✅ 学会了通过统计信息优化参数配置

#### 🔑 **关键要点回顾**
1. **参数配置 = 任务优化**：不同任务需要不同的参数组合
2. **成本控制 = 智能设计**：通过参数控制实现成本效益最优化
3. **统计分析 = 持续改进**：利用使用统计不断优化配置
4. **数据流理解 = 调试能力**：掌握完整的请求-响应流程

#### 🚀 **下一步**
在下一章中，我们将学习如何使用 Hooks 系统深入观察和调试 Agent 的执行过程！

---


## 第 4 章：高级调试 - 使用 Hooks 观察 Agent

在前几章中，我们构建了 Agent 并控制了其行为。现在，让我们学习如何使用 Hooks 系统深入观察和调试 Agent 的执行过程。

### 什么是 Hooks？
Hooks 是 Arkitect 框架提供的“钩子”机制，允许您在 Agent 执行的关键点插入自定义逻辑。它们就像“监听器”，帮助您：
- 🔍 **观察内部过程**：监控 LLM 调用、工具执行等
- 📊 **性能分析**：记录时间、token 使用等指标
- ⚠️ **错误处理**：捕获和干预异常
- 🔧 **自定义行为**：在特定点修改数据流

Hooks 是生产级应用的必备工具，尤其在调试复杂 Agent 时。

### Hooks 的核心价值
- 提供 Agent “黑箱”内部的可见性
- 支持异步操作，不影响性能
- 与 ContextInterruption 结合处理异常
- 易于扩展，用于日志、监控等

接下来，我们将学习四种主要 Hook 类型，并通过实际示例演示其用法。

### 四种主要 Hook 类型

Arkitect 提供了以下 Hook 类型，覆盖 Agent 执行的完整生命周期：

1. **PreLLMCallHook**：在调用 LLM 前触发
   - 用途：修改请求参数、记录输入
   - 时机：Context 准备发送消息给 LLM 前

2. **PostLLMCallHook**：LLM 调用后触发
   - 用途：分析响应、处理工具调用决定
   - 时机：收到 LLM 响应后

3. **PreToolCallHook**：工具调用前触发
   - 用途：验证工具参数、记录调用
   - 时机：决定调用工具前

4. **PostToolCallHook**：工具调用后触发
   - 用途：处理工具结果、错误恢复
   - 时机：工具执行完成后

**关键理解**：
- Hooks 通过 Context 的 `hooks` 参数注册
- 可以同时注册多个 Hooks
- Hooks 接收上下文参数，支持异步操作

**与其他概念的联系**：
- Hooks 可以访问 State 和 ArkMessage
- 与 ContextInterruption 结合处理异常
- 常用于分析 ArkChatRequest/Response

In [27]:
# 示例：创建简单的打印 Hook


# 示例：创建简单的打印 Hook
# 注意：Hook只能返回State，不能返回其它类型
# 注意：XXCallHook Class 下 xx_call 是空方法，需要重写


class DebugPreLLMHook(PreLLMCallHook):
    async def pre_llm_call(self, state: State) -> State:
        print("🔍 Pre LLM Hook: 请求发送中...")
        print(f"   消息数量: {len(state.messages)}")
        print(f"   消息: {state.messages}")
        return state  # 返回 state，支持修改


class DebugPostLLMHook(PostLLMCallHook):
    async def post_llm_call(self, state: State) -> State:
        print("📊 Post LLM Hook: 响应接收完成")
        if state.messages:
            response_message = state.messages[-1]  # 最新响应
            content = response_message.get("content", "")
            if not isinstance(content, str):
                content = str(content)
            print(f"   响应: {content}...")
        else:
            print("   无响应消息")
        return state


class DebugPreToolHook(PreToolCallHook):
    async def pre_tool_call(self, name: str, arguments: str, state: State) -> State:
        print(f"🛠️ Pre Tool Hook: 调用工具 {name}")
        print(f"   参数: {arguments}")
        return state  # 返回参数，支持修改


class DebugPostToolHook(PostToolCallHook):
    async def post_tool_call(
        self,
        name: str,
        arguments: str,
        response: Any,
        exception: Optional[Exception],
        state: State,
    ) -> State | Any:
        print(f"✅ Post Tool Hook: 工具 {name} 执行完成")
        if exception:
            print(f"❌ 异常: {str(exception)}")  # 处理异常，如 tool_ability_tester
            return "错误：工具调用失败"  # 自定义返回
        print(f"   结果预览: {str(response)[:50]}...")
        return state


print("✅ 调试 Hooks 已创建！")

✅ 调试 Hooks 已创建！


In [ ]:
async def create_hooked_agent():
    """创建带 Hooks 的 Agent"""

    # 创建 Context 并注册 Hooks
    MODEL = "doubao-seed-1-6-flash-250615"
    hooked_context = Context(
        model=MODEL,
        tools=[get_weather],
    )
    hooked_context.set_pre_llm_call_hook(DebugPreLLMHook())
    hooked_context.set_post_llm_call_hook(DebugPostLLMHook())
    hooked_context.set_pre_tool_call_hook(DebugPreToolHook())
    hooked_context.set_post_tool_call_hook(DebugPostToolHook())
    await hooked_context.init()

    print("✅ 带 Hooks 的 Agent 已创建！")
    return hooked_context


# 创建 Agent
hooked_context = await create_hooked_agent()

✅ 带 Hooks 的 Agent 已创建！


In [29]:
# 测试带 Hooks 的 Agent
test_question = "北京的天气如何？"

print("\n🧪 测试带 Hooks 的 Agent")
print("=" * 40)

completion = await hooked_context.completions.create(
    [{"role": "user", "content": test_question}], stream=True
)

print(f"👤 用户: {test_question}")
print("🤖 Agent: \n", end="")

async for chunk in completion:
    if hasattr(chunk, "choices") and chunk.choices:
        delta = chunk.choices[0].delta
        if hasattr(delta, "content") and delta.content:
            print(delta.content, end="")

print("\n\n💡 注意观察控制台输出的 Hook 日志！")


🧪 测试带 Hooks 的 Agent
👤 用户: 北京的天气如何？
🤖 Agent: 
🔍 Pre LLM Hook: 请求发送中...
   消息数量: 1
   消息: [{'role': 'user', 'content': '北京的天气如何？'}]
📊 Post LLM Hook: 响应接收完成
   响应: ...
🛠️ Pre Tool Hook: 调用工具 get_weather
   参数: {"city":"北京"}
🛠️ Pre Tool Hook: 调用工具 get_weather
   参数: {"city":"北京"}
✅ Post Tool Hook: 工具 get_weather 执行完成
   结果预览: meta=None content=[TextContent(type='text', text="...
🔍 Pre LLM Hook: 请求发送中...
   消息数量: 3
   消息: [{'role': 'user', 'content': '北京的天气如何？'}, {'content': '', 'role': 'assistant', 'function_call': None, 'tool_calls': [{'index': 0, 'id': 'call_z6bxgzm84yb8lexg81uf0wyg', 'function': {'arguments': '{"city":"北京"}', 'name': 'get_weather'}, 'type': 'function'}], 'audio': None, 'reasoning_content': None}, {'role': 'tool', 'tool_call_id': 'call_z6bxgzm84yb8lexg81uf0wyg', 'content': "([TextContent(type='text', text='北京今天的天气：晴天，气温 25°C，微风', annotations=None, meta=None)], {'result': '北京今天的天气：晴天，气温 25°C，微风'})"}]
北京今天的天气：晴天，气温 25°C，微风📊 Post LLM Hook: 响应接收完成
   响应: 北京今天的天气：晴天，气温 25°C，

#### 高级 Hook：性能监控与异常处理

在实际工程中，Hook 常用于监控工具调用的性能、捕获异常并与 Agent 的中断机制结合。  
这些能力本质上是对前面 Context、工具、异常机制的灵活组合。

**常见场景：**
- 统计每次工具调用的耗时
- 捕获并记录工具调用异常
- 遇到严重异常时优雅中断对话

**实现思路：**
- 在 `PreToolCallHook` 记录开始时间，在 `PostToolCallHook` 计算耗时并输出日志。
- 在 `PostToolCallHook` 检查 `tool_call.exception`，如有异常则记录或抛出 `ContextInterruption`。
- 这些 Hook 可与前面定义的 Agent/工具无缝结合，无需额外复杂代码。

**示例代码片段：**
```python
class PerfAndErrorHook(PostToolCallHook):
    async def __call__(self, context, tool_call, result):
        if tool_call.exception:
            print(f"工具调用异常: {tool_call.exception}")
        else:
            print(f"工具 {tool_call.name} 正常完成")
```
将此 Hook 加入 Context 后，所有工具调用都会被自动监控和异常捕获。

#### Hook 在自动化测试中的应用

Hook 还可以用于自动化测试，记录每次对话的输入输出，或断言 Agent 行为是否符合预期。

**常见场景：**
- 自动记录对话内容，便于回归测试
- 在测试用例中断言输出内容是否符合预期

**推荐实现方式：**
- 自定义一个 Hook，将输入输出保存到列表或日志
- 在测试用例中直接断言这些记录

In [30]:
class TestRecorderHook(PostLLMCallHook):
    async def post_llm_call(self, state: State) -> State:
        print(f"【测试记录】输入: {state.messages}，输出: {state.messages[-1]}")
        return state


# 在测试用例中断言输出
async def test_agent_reply():
    print("🧪 自动化测试 Hook 演示")
    print("=" * 50)

    from arkitect.core.component.context.context import Context

    context = Context(model="doubao-seed-1-6-flash-250615")
    await context.init()
    context.set_post_llm_call_hook(TestRecorderHook())

    # 发送消息并断言输出内容
    completion = await context.completions.create(
        [{"role": "user", "content": "你好，今天的天气如何"}], stream=False
    )
    reply = completion.choices[0].message.content
    print(f"AI 回复: {reply}")
    assert "人工智能" in reply and "AI" in reply, "回复内容不符合预期"


# 运行测试
await test_agent_reply()

🧪 自动化测试 Hook 演示
【测试记录】输入: [{'role': 'user', 'content': '你好，今天的天气如何'}, {'content': '要查询今天的天气，你可以通过以下几种方式：\n### 1. 手机天气应用\n打开手机自带的天气应用程序（如苹果的“天气”、安卓的天气相关应用），一般会自动定位并显示当地实时天气。\n\n### 2. 搜索引擎查询\n打开浏览器，在搜索框中输入“[你所在城市]今天天气”，比如“北京今天天气”，点击搜索结果就能看到详细的天气信息（包括温度、阴晴状况、风力等）。\n\n### 3. 天气网站\n访问专业的天气网站，如中国天气网（https://www.weather.com.cn/），在网站上输入所在城市名称，即可获取今天的天气详情。', 'role': 'assistant', 'function_call': None, 'tool_calls': None, 'audio': None, 'reasoning_content': '用户现在想知道今天的天气，但我没办法直接获取实时天气呢。用户可以通过打开手机的天气应用，或者在浏览器里搜索当地的天气查询网站，这样就能查到准确的今天当地天气情况啦。'}]，输出: {'content': '要查询今天的天气，你可以通过以下几种方式：\n### 1. 手机天气应用\n打开手机自带的天气应用程序（如苹果的“天气”、安卓的天气相关应用），一般会自动定位并显示当地实时天气。\n\n### 2. 搜索引擎查询\n打开浏览器，在搜索框中输入“[你所在城市]今天天气”，比如“北京今天天气”，点击搜索结果就能看到详细的天气信息（包括温度、阴晴状况、风力等）。\n\n### 3. 天气网站\n访问专业的天气网站，如中国天气网（https://www.weather.com.cn/），在网站上输入所在城市名称，即可获取今天的天气详情。', 'role': 'assistant', 'function_call': None, 'tool_calls': None, 'audio': None, 'reasoning_content': '用户现在想知道今天的天气，但我没办法直接获取实时天气呢。用户可以通过打开手机的天气应用，或者在浏览器里搜索当地的天气查询网站，这样就能查到准确的

AssertionError: 回复内容不符合预期

### Hook 最佳实践
- **日志记录**：使用 Hooks 记录所有关键事件
- **性能优化**：监控耗时长的操作
- **错误恢复**：实现自动重试机制
- **不要阻塞**：保持 Hooks 异步且高效

### 🎯 第 4 章总结

通过本章，您掌握了 Hooks 系统：
- 理解了四种 Hook 类型及其生命周期
- 学会了创建和应用自定义 Hooks
- 掌握了性能监控和错误处理技巧
- 了解了 Hooks 与 ContextInterruption 的结合

**下一步**： Cookbook 即将结束，请查看 Q&A 和结语！

## Q&A：常见问题和解决方案

这里汇总了使用 Arkitect 框架的常见问题及解决方案。

1. **Q: 如何处理工具调用异常？**  
   A: 使用 PostToolCallHook 捕获异常，或在工具函数中返回错误字符串。参考第4章错误处理示例。

2. **Q: temperature 参数如何选择？**  
   A: 工具任务用 0.1-0.3，创意任务用 0.7-1.0。参考第3章实验。

3. **Q: 为什么 stream=True 时无 usage？**  
   A: 流式模式不返回 usage，使用 stream=False 获取。参考第3章说明。

4. **Q: 如何自定义 System Prompt？**  
   A: 在 messages 列表中添加 role="system" 消息。参考第1章示例。

5. **Q: Agent 内存占用高如何优化？**  
   A: 定期清除 context.state.messages，或使用 max_tokens 限制输出。

6. **Q: Hooks 影响性能吗？**  
   A: 轻量 Hooks 无影响，避免在 Hooks 中添加重计算。

7. **Q: 如何测试多模型兼容？**  
   A: 在 Context 初始化时切换 model 参数，并运行相同测试。

更多问题？欢迎在 GitHub Issues 反馈！

---


## 结语：您的 Arkitect 之旅

恭喜！您已完成 Arkitect Cookbook 的核心内容。从基础 Context 到高级 Hooks，您现在具备构建强大 AI Agent 的能力。

### 关键收获
- **基础**：掌握 Context、State 和消息管理
- **构建**：创建工具驱动的 Agent
- **控制**：通过参数优化行为
- **调试**：使用 Hooks 监控一切

### 行动号召
- 实践：基于本教程构建您的第一个生产 Agent
- 扩展：探索更多模型和工具集成
- 贡献：分享您的经验或改进建议

感谢您的学习！如果有疑问，随时重温 Q&A 或联系我们。开始构建属于您自己的Agent吧！🚀